In [ ]:
!pip install gradio transformers PyPDF2

In [ ]:
import gradio as gr
import PyPDF2
from transformers import AutoModelForCausalLM, AutoTokenizer
import random


In [ ]:

# Load the model
model_name = "ibm-granite/granite-3.3-2b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [ ]:
# Function to generate responses from the model
def generate_response(prompt, max_length=500):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Scenario 1: Requirement Upload and Classification
def classify_requirements(pdf_file):
    try:
        reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"

        # Classify sentences into SDLC phases
        classified = {
            "Requirements": [],
            "Design": [],
            "Development": [],
            "Testing": [],
            "Deployment": []
        }

        sentences = [s.strip() for s in text.split('.') if s.strip()]
        for sentence in sentences:
            prompt = f"Classify this software requirement into SDLC phase (Requirements, Design, Development, Testing, Deployment): {sentence}"
            phase = generate_response(prompt, 50).strip()

            # Clean up the response
            phase = phase.split('\n')[0].split('.')[0].strip()
            if phase in classified:
                classified[phase].append(sentence)
            else:
                classified["Requirements"].append(sentence)

        return classified

    except Exception as e:
        return f"Error processing PDF: {str(e)}"

# Scenario 2: AI Code Generator
def generate_code_from_story(user_story):
    prompt = f"""Generate production-ready Python code based on this user story:
    {user_story}

    Include proper error handling and comments. Return only the code block."""
    return generate_response(prompt)

# Scenario 3: Bug Fixer
def fix_buggy_code(buggy_code):
    prompt = f"""Find and fix all bugs in this code:
    {buggy_code}

    Explain the fixes and return the corrected code with comments."""
    return generate_response(prompt)

# Scenario 4: Test Case Generator
def generate_test_cases(code_or_req):
    prompt = f"""Generate comprehensive pytest test cases for the following code or requirement:
    {code_or_req}

    Include edge cases and return properly formatted pytest code."""
    return generate_response(prompt)

# Scenario 5: Code Summarizer
def summarize_code(source_code):
    prompt = f"""Provide a detailed summary of this code's functionality:
    {source_code}

    Include: purpose, main functions, inputs/outputs, and any important notes."""
    return generate_response(prompt)

# Scenario 6: Floating AI Chatbot Assistant
def chatbot_response(query):
    prompt = f"""You are an expert SDLC assistant. Help with this query:
    {query}

    Provide detailed, actionable advice."""
    return generate_response(prompt)



In [ ]:
# Enhanced CSS with modern design
css = """
/* Global Styles */
.gradio-container {
    max-width: 1400px !important;
    margin: 0 auto !important;
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif !important;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    min-height: 100vh !important;
}

/* Header Styling */
.header-title {
    background: linear-gradient(90deg, #ffffff 0%, #f8f9fa 100%) !important;
    padding: 25px !important;
    border-radius: 15px !important;
    margin-bottom: 30px !important;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1) !important;
    text-align: center !important;
    border: 1px solid rgba(255, 255, 255, 0.2) !important;
}

.header-title h1 {
    background: linear-gradient(45deg, #667eea, #764ba2) !important;
    background-clip: text !important;
    -webkit-background-clip: text !important;
    -webkit-text-fill-color: transparent !important;
    font-size: 2.5rem !important;
    font-weight: 700 !important;
    margin: 0 !important;
    text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.1) !important;
}

.header-title p {
    color: #6c757d !important;
    font-size: 1.2rem !important;
    margin: 10px 0 0 0 !important;
    font-weight: 500 !important;
}

/* Tab Styling - Navigation Bar Look */
.tab-nav {
    background: rgba(255, 255, 255, 0.95) !important;
    backdrop-filter: blur(10px) !important;
    border-radius: 15px !important;
    padding: 10px !important;
    margin-bottom: 25px !important;
    box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1) !important;
    border: 1px solid rgba(255, 255, 255, 0.3) !important;
}

.tab-nav button {
    background: linear-gradient(145deg, #f8f9fa, #e9ecef) !important;
    color: #495057 !important;
    border: none !important;
    padding: 12px 24px !important;
    margin: 5px !important;
    border-radius: 25px !important;
    font-weight: 600 !important;
    font-size: 14px !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 4px 15px rgba(0, 0, 0, 0.1) !important;
    cursor: pointer !important;
}

.tab-nav button:hover {
    background: linear-gradient(145deg, #667eea, #764ba2) !important;
    color: white !important;
    transform: translateY(-2px) !important;
    box-shadow: 0 6px 20px rgba(102, 126, 234, 0.4) !important;
}

.tab-nav button.selected {
    background: linear-gradient(145deg, #667eea, #764ba2) !important;
    color: white !important;
    box-shadow: 0 6px 20px rgba(102, 126, 234, 0.4) !important;
}

/* Content Area Styling */
.tab-content {
    background: rgba(255, 255, 255, 0.95) !important;
    backdrop-filter: blur(15px) !important;
    border-radius: 20px !important;
    padding: 30px !important;
    box-shadow: 0 12px 40px rgba(0, 0, 0, 0.15) !important;
    border: 1px solid rgba(255, 255, 255, 0.3) !important;
    margin-bottom: 20px !important;
}

/* Input and Output Box Styling */
.input-box, .output-box {
    background: linear-gradient(145deg, #f8f9fa, #e9ecef) !important;
    border: 2px solid rgba(102, 126, 234, 0.2) !important;
    border-radius: 15px !important;
    padding: 20px !important;
    margin: 15px 0 !important;
    box-shadow: inset 0 2px 10px rgba(0, 0, 0, 0.05) !important;
    font-family: 'Fira Code', 'Consolas', monospace !important;
    font-size: 14px !important;
    line-height: 1.6 !important;
}

.input-box:focus, .output-box:focus {
    border-color: #667eea !important;
    box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1) !important;
    outline: none !important;
}

/* Text Areas */
textarea {
    background: linear-gradient(145deg, #f8f9fa, #e9ecef) !important;
    border: 2px solid rgba(102, 126, 234, 0.2) !important;
    border-radius: 15px !important;
    padding: 20px !important;
    font-family: 'Fira Code', 'Consolas', monospace !important;
    font-size: 14px !important;
    color: #2d3748 !important;
    resize: vertical !important;
    min-height: 120px !important;
}

textarea:focus {
    border-color: #667eea !important;
    box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1) !important;
    outline: none !important;
}

/* File Upload Styling */
.file-upload {
    background: linear-gradient(145deg, #f8f9fa, #e9ecef) !important;
    border: 3px dashed rgba(102, 126, 234, 0.3) !important;
    border-radius: 15px !important;
    padding: 30px !important;
    text-align: center !important;
    transition: all 0.3s ease !important;
    cursor: pointer !important;
}

.file-upload:hover {
    border-color: #667eea !important;
    background: linear-gradient(145deg, #e9ecef, #dee2e6) !important;
    transform: translateY(-2px) !important;
}

/* Button Styling */
.btn-primary {
    background: linear-gradient(145deg, #667eea, #764ba2) !important;
    border: none !important;
    color: white !important;
    padding: 15px 30px !important;
    font-size: 16px !important;
    font-weight: 600 !important;
    border-radius: 25px !important;
    cursor: pointer !important;
    transition: all 0.3s ease !important;
    box-shadow: 0 6px 20px rgba(102, 126, 234, 0.3) !important;
    margin: 10px 5px !important;
}

.btn-primary:hover {
    background: linear-gradient(145deg, #5a6fd8, #6c5ce7) !important;
    transform: translateY(-3px) !important;
    box-shadow: 0 8px 25px rgba(102, 126, 234, 0.4) !important;
}

.btn-primary:active {
    transform: translateY(-1px) !important;
    box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3) !important;
}

/* Code Block Styling */
.code-container {
    background: linear-gradient(145deg, #2d3748, #1a202c) !important;
    border-radius: 15px !important;
    padding: 25px !important;
    margin: 15px 0 !important;
    box-shadow: 0 8px 25px rgba(0, 0, 0, 0.2) !important;
    border: 1px solid rgba(255, 255, 255, 0.1) !important;
}

.code-container pre {
    color: #e2e8f0 !important;
    font-family: 'Fira Code', 'Consolas', monospace !important;
    font-size: 14px !important;
    line-height: 1.6 !important;
    margin: 0 !important;
    white-space: pre-wrap !important;
    word-wrap: break-word !important;
}

/* JSON Output Styling */
.json-output {
    background: linear-gradient(145deg, #f8f9fa, #e9ecef) !important;
    border-radius: 15px !important;
    padding: 20px !important;
    max-height: 400px !important;
    overflow-y: auto !important;
    font-family: 'Fira Code', 'Consolas', monospace !important;
    font-size: 13px !important;
    color: #2d3748 !important;
    border: 2px solid rgba(102, 126, 234, 0.2) !important;
}

/* Labels */
label {
    color: #4a5568 !important;
    font-weight: 600 !important;
    font-size: 16px !important;
    margin-bottom: 8px !important;
    display: block !important;
}

/* Scrollbar Styling */
::-webkit-scrollbar {
    width: 8px !important;
}

::-webkit-scrollbar-track {
    background: rgba(0, 0, 0, 0.1) !important;
    border-radius: 10px !important;
}

::-webkit-scrollbar-thumb {
    background: linear-gradient(145deg, #667eea, #764ba2) !important;
    border-radius: 10px !important;
}

::-webkit-scrollbar-thumb:hover {
    background: linear-gradient(145deg, #5a6fd8, #6c5ce7) !important;
}

/* Responsive Design */
@media (max-width: 768px) {
    .header-title h1 {
        font-size: 2rem !important;
    }

    .tab-nav button {
        padding: 10px 15px !important;
        font-size: 12px !important;
        margin: 3px !important;
    }

    .tab-content {
        padding: 20px !important;
    }
}

/* Animation Effects */
@keyframes fadeIn {
    from { opacity: 0; transform: translateY(20px); }
    to { opacity: 1; transform: translateY(0); }
}

.fade-in {
    animation: fadeIn 0.6s ease-out !important;
}

/* Glassmorphism Effect */
.glass {
    background: rgba(255, 255, 255, 0.25) !important;
    backdrop-filter: blur(10px) !important;
    border: 1px solid rgba(255, 255, 255, 0.18) !important;
}
"""



In [ ]:
# Create Enhanced Gradio Interface
with gr.Blocks(css=css, title="SmartSDLC - AI-Enhanced SDLC", theme=gr.themes.Soft()) as app:
    # Header
    gr.HTML("""
    <div class="header-title fade-in">
        <h1>🚀 SmartSDLC</h1>
        <p>AI-Enhanced Software Development Lifecycle with IBM Granite</p>
    </div>
    """)

    with gr.Tabs(elem_classes="tab-nav") as tabs:
        with gr.Tab("📋 Requirements Classifier", elem_classes="tab-content"):
            gr.Markdown("### Upload and Classify Software Requirements")
            with gr.Row():
                with gr.Column(scale=1):
                    pdf_input = gr.File(
                        label="📄 Upload PDF Requirements",
                        file_types=[".pdf"],
                        elem_classes="file-upload"
                    )
                    classify_button = gr.Button(
                        "🔍 Classify Requirements",
                        variant="primary",
                        elem_classes="btn-primary"
                    )
                with gr.Column(scale=2):
                    classification_output = gr.JSON(
                        label="📊 Classified Requirements by SDLC Phase",
                        elem_classes="json-output"
                    )

            classify_button.click(
                classify_requirements,
                inputs=pdf_input,
                outputs=classification_output
            )

        with gr.Tab("💻 Code Generator", elem_classes="tab-content"):
            gr.Markdown("### Generate Code from User Stories")
            with gr.Row():
                with gr.Column(scale=1):
                    user_story_input = gr.Textbox(
                        label="📝 Enter User Story",
                        placeholder="As a user, I want to...",
                        lines=5,
                        elem_classes="input-box"
                    )
                    generate_code_button = gr.Button(
                        "⚡ Generate Code",
                        variant="primary",
                        elem_classes="btn-primary"
                    )
                with gr.Column(scale=2):
                    code_output = gr.Code(
                        label="🐍 Generated Python Code",
                        language="python",
                        elem_classes="code-container"
                    )

            generate_code_button.click(
                generate_code_from_story,
                inputs=user_story_input,
                outputs=code_output
            )

        with gr.Tab("🐛 Bug Fixer", elem_classes="tab-content"):
            gr.Markdown("### Identify and Fix Code Bugs")
            with gr.Row():
                with gr.Column(scale=1):
                    buggy_code_input = gr.Textbox(
                        label="🔍 Enter Buggy Code",
                        placeholder="Paste your buggy code here...",
                        lines=8,
                        elem_classes="input-box"
                    )
                    fix_code_button = gr.Button(
                        "🔧 Fix Code",
                        variant="primary",
                        elem_classes="btn-primary"
                    )
                with gr.Column(scale=2):
                    fixed_code_output = gr.Code(
                        label="✅ Fixed Code with Explanations",
                        language="python",
                        elem_classes="code-container"
                    )

            fix_code_button.click(
                fix_buggy_code,
                inputs=buggy_code_input,
                outputs=fixed_code_output
            )

        with gr.Tab("🧪 Test Generator", elem_classes="tab-content"):
            gr.Markdown("### Generate Comprehensive Test Cases")
            with gr.Row():
                with gr.Column(scale=1):
                    functional_code_input = gr.Textbox(
                        label="📋 Enter Code or Requirements",
                        placeholder="Paste your functional code here...",
                        lines=8,
                        elem_classes="input-box"
                    )
                    generate_test_cases_button = gr.Button(
                        "🧪 Generate Tests",
                        variant="primary",
                        elem_classes="btn-primary"
                    )
                with gr.Column(scale=2):
                    test_case_output = gr.Code(
                        label="🎯 Generated Pytest Test Cases",
                        language="python",
                        elem_classes="code-container"
                    )

            generate_test_cases_button.click(
                generate_test_cases,
                inputs=functional_code_input,
                outputs=test_case_output
            )

        with gr.Tab("📖 Code Summarizer", elem_classes="tab-content"):
            gr.Markdown("### Analyze and Summarize Code Functionality")
            with gr.Row():
                with gr.Column(scale=1):
                    source_code_input = gr.Textbox(
                        label="📄 Enter Source Code",
                        placeholder="Paste your source code here...",
                        lines=10,
                        elem_classes="input-box"
                    )
                    summarize_code_button = gr.Button(
                        "📊 Summarize Code",
                        variant="primary",
                        elem_classes="btn-primary"
                    )
                with gr.Column(scale=2):
                    summary_output = gr.Textbox(
                        label="📋 Detailed Code Summary",
                        lines=15,
                        elem_classes="output-box"
                    )

            summarize_code_button.click(
                summarize_code,
                inputs=source_code_input,
                outputs=summary_output
            )

        with gr.Tab("🤖 AI Assistant", elem_classes="tab-content"):
            gr.Markdown("### Your Personal SDLC Expert")
            with gr.Row():
                with gr.Column(scale=1):
                    user_query_input = gr.Textbox(
                        label="💬 Ask Your SDLC Question",
                        placeholder="How can I improve my software development process?",
                        lines=4,
                        elem_classes="input-box"
                    )
                    ask_button = gr.Button(
                        "🚀 Get Expert Advice",
                        variant="primary",
                        elem_classes="btn-primary"
                    )
                with gr.Column(scale=2):
                    chatbot_output = gr.Textbox(
                        label="🎯 Expert Response",
                        lines=12,
                        elem_classes="output-box"
                    )

            ask_button.click(
                chatbot_response,
                inputs=user_query_input,
                outputs=chatbot_output
            )

    # Footer
    gr.HTML("""
    <div style="text-align: center; margin-top: 40px; padding: 20px;
                background: rgba(255, 255, 255, 0.1); border-radius: 15px;
                backdrop-filter: blur(10px); color: white;">
        <p style="margin: 0; font-size: 14px;">
            🌟 Powered by IBM Granite AI | Built with ❤️ for Developers
        </p>
    </div>
    """)



In [ ]:
# Launch the app
if __name__ == "__main__":
    app.launch(debug=True, share=True)